# Deltasharing med skyporten

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../..") # go to parent dir

## Set enviroment variable fra Maskinporten/Skyporten

Om du ikke har opprettet en integrasjon i Maskinporten kan du hente det i forenklet onboarding(https://onboarding.test.maskinporten.no)

* MASKINPORTEN_KID=keyid
* MASKINPORTEN_INTEGRATION_ID=uuid

Og legg tilhørende privatnøkkelen under `/creds/maskinporten.pem`


### The config object



In [ ]:
skyporten_config = {
    "scope": "foo:adminenheter",
    "bucket_name": "skyporten-222222222",
    "filename": "config.share",
    "resource": "https://skyporten.fooet",
    "GOOGLE_PROJECT_ID": "dataplattform-sandbox-9999",
    "SERVICE_ACCOUNT_EMAIL": "skyportenstorageconsumer@dataplattform-sandbox-9999.iam.gserviceaccount.com",
    "POOL_PROVIDER": "testskyporten",
    "PROVIDER_NAME": "testskyporten",
    "PROJECT_NUMBER": "99999999999999"
}

In [ ]:
import skyporten
import shareconsume
from shareconsume.in_mem_credentials import ShareConfig,pandas_from_share

In [ ]:
credentials = skyporten.get_bucket_file(
    config=skyporten_config,
)

In [ ]:
share_config = ShareConfig(share="adm-enheter", schema="silver", name="kommuner")

In [ ]:
df = pandas_from_share(credentials=credentials, share_config=share_config)
df.show(5)

# Setup

In [ ]:
%sh pip install delta-sharing

In [ ]:
import delta_sharing


I manually created a volume to write files, since shared clusters have limited file system permissions

In [ ]:
deltashare = dbutils.secrets.get("skyportensuppeverket", "unitsdeltashare")
deltasharepath = "/Volumes/training/suppeverket/skyportensupperverket/skyportendeltashare/skyporten_config.share"
dbutils.fs.put(deltasharepath, deltashare, True)

In [ ]:
client = delta_sharing.SharingClient(deltasharepath)
client.list_all_tables()

In [ ]:
profile_path = deltasharepath
share_name = 'units'
schema_name= 'silver'
table_name = 'municipalities'
share_uc_path = f"{profile_path}#{share_name}.{schema_name}.{table_name}"
print("share_uc_path", share_uc_path)
kommuner = (
    spark.read.format("deltaSharing")
    .load(share_uc_path)
    .limit(10)
)
kommuner.display()